In [48]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
data = pd.read_csv("movie_recommender (1).csv")
data.head()

,user_id,movie_id,title,genres,rating
0,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4
1,1,2,Jumanji (1995),Adventure|Children|Fantasy,3
2,1,3,Grumpier Old Men (1995),Comedy|Romance,2
3,1,4,Waiting to Exhale (1995),Comedy|Drama|Romance,5
4,2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5


## Create a Pivot Table

In [50]:
user_items_matrix = data.pivot_table(index='user_id', columns='movie_id', values='rating')
user_items_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
user_id,,,,,,,,,,,,,,,,,,,,,,
1,4.0,3.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
user_items_matrix = user_items_matrix.fillna(0)

In [52]:
user_items_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
user_id,,,,,,,,,,,,,,,,,,,,,,
1,4.0,3.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
def find_similar_user(user_id, user_item_matrix, n_users = 5):
    similarities = cosine_similarity(user_item_matrix)
    similarities_df = pd.DataFrame(similarities, index=user_item_matrix.index, columns=user_item_matrix.index)

    similar_users = similarities_df[user_id].sort_values(ascending=False).index[1:n_users + 1]
    return similar_users



In [54]:
similar_users = find_similar_user(2, user_items_matrix)
similar_users

Index([3, 1, 4, 5, 6], dtype='int64', name='user_id')

In [55]:
def recommend_movies(user_id, user_items_matrix, n_recom =5):
    similar_users = find_similar_user(user_id, user_items_matrix)
    desired_ratings = user_items_matrix.loc[user_id]
    unseen_movies = desired_ratings[desired_ratings==0].index

    recommended_movies = user_items_matrix.loc[similar_users, unseen_movies].mean(axis=0)
    return recommended_movies.sort_values(ascending=False).head(n_recom)


In [56]:
movies = recommend_movies(1, user_items_matrix)
movies

movie_id
10    1.0
13    1.0
5     0.8
6     0.8
16    0.8
dtype: float64